In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress
import time
import pandas as pd

In [2]:
#Location of files containing the traces
#Modify appropriately
trace_address = 'traces/trace.npy'
nonce_address = 'traces/nonce_text.npy'
plaintext_address = 'traces/plain_text.npy'
key_address = 'traces/key.npy'
header_address = 'traces/header_text.npy'

#Load the traces, plaintext, nonce, key, headers into numpy arrays for processing
trace_array = np.load(trace_address)
nonce_array = np.load(nonce_address)
pt_array = np.load(plaintext_address)
key_array = np.load(key_address)
header_array = np.load(header_address)

#Type formatting to resolve some numpy errors
nonce_array = nonce_array.astype(np.int64)

print("Number of traces:", trace_array.shape[0])
print("Number of points:", trace_array.shape[1], "\n\n")

Number of traces: 1000
Number of points: 100000 




In [3]:
print(key_array)

[[ 30  76  31  50 180 244 129  74 151 132 200 230  27 171  21 170]]


In [4]:
# Round constants*
GIFT_RC = [
    0x01, 0x03, 0x07, 0x0F, 0x1F, 0x3E, 0x3D, 0x3B, 0x37, 0x2F,
    0x1E, 0x3C, 0x39, 0x33, 0x27, 0x0E, 0x1D, 0x3A, 0x35, 0x2B,
    0x16, 0x2C, 0x18, 0x30, 0x21, 0x02, 0x05, 0x0B, 0x17, 0x2E,
    0x1C, 0x38, 0x31, 0x23, 0x06, 0x0D, 0x1B, 0x36, 0x2D, 0x1A
]

#Hamming weight - Counts the number of 1's
HW = [bin(n).count("1") for n in range(0, 256)]

#Row Permutation Operation
def rowperm(S, B0_pos, B1_pos, B2_pos, B3_pos):
    T=0
    for b in range(8):
        T |= ((S>>(4*b+0))&0x1)<<(b + 8*B0_pos)
        T |= ((S>>(4*b+1))&0x1)<<(b + 8*B1_pos)
        T |= ((S>>(4*b+2))&0x1)<<(b + 8*B2_pos)
        T |= ((S>>(4*b+3))&0x1)<<(b + 8*B3_pos)
    return T

#Generating the intermediate value based on input and keyguesses
#The intermediate is correlated with the power traces to obtain the correct key
#This implementation guesses one bytes of the key at a time.  
def intermediate(P, keyguess, key_index):
    #P: Plain Text 
    #keyguess : Part of U or V: (W2 || W3) or (W6 || W7) that needs to be guessed
    #keyindex : Which byte of U & V is being attacked
    
    S = [0,0,0,0]
    
    # ===SubCells=== #
    S[0] = (P[ 0]<<24) | (P[ 1]<<16) | (P[ 2]<<8) | P[ 3]
    S[1] = (P[ 4]<<24) | (P[ 5]<<16) | (P[ 6]<<8) | P[ 7]
    S[2] = (P[ 8]<<24) | (P[ 9]<<16) | (P[10]<<8) | P[11]
    S[3] = (P[12]<<24) | (P[13]<<16) | (P[14]<<8) | P[15]

    S[1] ^= S[0] & S[2]
    S[0] ^= S[1] & S[3]
    S[2] ^= S[0] | S[1]
    S[3] ^= S[2]
    S[1] ^= S[3]
    S[3] ^= 0xffffffff
    S[2] ^= S[0] & S[1]

    T = S[0]
    S[0] = S[3]
    S[3] = T

    # ===PermBits=== #
    S[0] = rowperm(S[0],0,3,2,1)
    S[1] = rowperm(S[1],1,0,3,2)
    S[2] = rowperm(S[2],2,1,0,3)
    S[3] = rowperm(S[3],3,2,1,0)

    # ===AddRoundKey=== #
    
    #keyindex = 4: gives W2[0]
    if key_index == 4:        
        intermediate = ((S[2] >>  0) & 0b11111111) ^ keyguess
    #keyindex = 5: gives W2[1]
    elif key_index == 5:    
        intermediate = ((S[2] >>  8) & 0b11111111) ^ keyguess
    #keyindex = 6: gives W3[0]
    elif key_index == 6:    
        intermediate = ((S[2] >>  16) & 0b11111111) ^ keyguess
    #keyindex = 7: gives W3[1]
    elif key_index == 7:    
        intermediate = ((S[2] >>  24) & 0b11111111) ^ keyguess
    #keyindex = 12: gives W6[0]
    elif key_index == 12:    
        intermediate = ((S[1] >>  0) & 0b11111111) ^ keyguess
    #keyindex = 13: gives W6[1]
    elif key_index == 13:    
        intermediate = ((S[1] >>  8) & 0b11111111) ^ keyguess
    #keyindex = 14: gives W7[0]
    elif key_index == 14:    
        intermediate = ((S[1] >>  16) & 0b11111111) ^ keyguess
    #keyindex = 15: gives W7[1]
    elif key_index == 15:    
        intermediate = ((S[1] >>  24) & 0b11111111) ^ keyguess
    
    return intermediate
    
#Conversion function - to resolve type formatting errors    
def conversion(P):
    Pdash = [0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00]
    for i in range(16):
        Pdash[i] = P[i]
    return Pdash
    

In [5]:
numtraces = trace_array.shape[0]  #Number of traces
numpoints = trace_array.shape[1]  #Number of points in a trace

#Points within which the correlating point lies in 
start_point = 0                   #Starting Point: Point from which the correlating point might lie in
end_point = 5000                  #Ending Point: Point w which the correlating point might lie in

crvs = np.zeros((256, numpoints))
#Hamming Weight Matrix
HW_matrix = np.zeros((numtraces, 256), dtype=np.uint8)

#Stores the correlation value for each key guess
temp = np.zeros(256)

for kguess in range(0, 256):
    
    # Generate the hamming weight for each possible keybyte
    for trace_no in range(numtraces):
        nonce_temp = conversion(nonce_array[trace_no])
        iv = intermediate(nonce_temp, kguess, 4)
        HW_matrix[trace_no, kguess] = HW[iv]

    # correlate the trace with the hamming weights        
    hw = HW_matrix[:, kguess]
    for point in range(start_point, end_point):
        trc = trace_array[:, point]
        crvs[kguess, point] = np.abs(linregress(hw, trc).slope)

    #Finding the point in the trace which has maximum correlation        
    temp[kguess] = np.max(crvs[kguess])

#Finding the key guess that corresponds to maximum correlation
key_estimated = np.argmax(temp)
print("R4 = ",key_estimated, "  Correlation = ", temp[key_estimated])


R4 =  31   Correlation =  0.003208892191261737


In [6]:
numtraces = trace_array.shape[0]  #Number of traces
numpoints = trace_array.shape[1]  #Number of points in a trace

#Points within which the correlating point lies in 
start_point = 0                   #Starting Point: Point from which the correlating point might lie in
end_point = 5000                  #Ending Point: Point w which the correlating point might lie in

crvs = np.zeros((256, numpoints))
#Hamming Weight Matrix
HW_matrix = np.zeros((numtraces, 256), dtype=np.uint8)

#Stores the correlation value for each key guess
temp = np.zeros(256)

for kguess in range(0, 256):
    
    # Generate the hamming weight for each possible keybyte
    for trace_no in range(numtraces):
        nonce_temp = conversion(nonce_array[trace_no])
        iv = intermediate(nonce_temp, kguess, 5)
        HW_matrix[trace_no, kguess] = HW[iv]

    # correlate the trace with the hamming weights        
    hw = HW_matrix[:, kguess]
    for point in range(start_point, end_point):
        trc = trace_array[:, point]
        crvs[kguess, point] = np.abs(linregress(hw, trc).slope)

    #Finding the point in the trace which has maximum correlation        
    temp[kguess] = np.max(crvs[kguess])

#Finding the key guess that corresponds to maximum correlation
key_estimated = np.argmax(temp)
print("R5 = ",key_estimated, "  Correlation = ", temp[key_estimated])


R5 =  0   Correlation =  0.002527467012681699


In [7]:
numtraces = trace_array.shape[0]  #Number of traces
numpoints = trace_array.shape[1]  #Number of points in a trace

#Points within which the correlating point lies in 
start_point = 0                   #Starting Point: Point from which the correlating point might lie in
end_point = 5000                  #Ending Point: Point w which the correlating point might lie in

crvs = np.zeros((256, numpoints))
#Hamming Weight Matrix
HW_matrix = np.zeros((numtraces, 256), dtype=np.uint8)

#Stores the correlation value for each key guess
temp = np.zeros(256)

for kguess in range(0, 256):
    
    # Generate the hamming weight for each possible keybyte
    for trace_no in range(numtraces):
        nonce_temp = conversion(nonce_array[trace_no])
        iv = intermediate(nonce_temp, kguess, 6)
        HW_matrix[trace_no, kguess] = HW[iv]

    # correlate the trace with the hamming weights        
    hw = HW_matrix[:, kguess]
    for point in range(start_point, end_point):
        trc = trace_array[:, point]
        crvs[kguess, point] = np.abs(linregress(hw, trc).slope)

    #Finding the point in the trace which has maximum correlation        
    temp[kguess] = np.max(crvs[kguess])

#Finding the key guess that corresponds to maximum correlation
key_estimated = np.argmax(temp)
print("R6 = ",key_estimated, "  Correlation = ", temp[key_estimated])


R6 =  56   Correlation =  0.002566231630705503


In [8]:
numtraces = trace_array.shape[0]  #Number of traces
numpoints = trace_array.shape[1]  #Number of points in a trace

#Points within which the correlating point lies in 
start_point = 0                   #Starting Point: Point from which the correlating point might lie in
end_point = 5000                  #Ending Point: Point w which the correlating point might lie in

crvs = np.zeros((256, numpoints))
#Hamming Weight Matrix
HW_matrix = np.zeros((numtraces, 256), dtype=np.uint8)

#Stores the correlation value for each key guess
temp = np.zeros(256)

for kguess in range(0, 256):
    
    # Generate the hamming weight for each possible keybyte
    for trace_no in range(numtraces):
        nonce_temp = conversion(nonce_array[trace_no])
        iv = intermediate(nonce_temp, kguess, 7)
        HW_matrix[trace_no, kguess] = HW[iv]

    # correlate the trace with the hamming weights        
    hw = HW_matrix[:, kguess]
    for point in range(start_point, end_point):
        trc = trace_array[:, point]
        crvs[kguess, point] = np.abs(linregress(hw, trc).slope)

    #Finding the point in the trace which has maximum correlation        
    temp[kguess] = np.max(crvs[kguess])

#Finding the key guess that corresponds to maximum correlation
key_estimated = np.argmax(temp)
print("R7 = ",key_estimated, "  Correlation = ", temp[key_estimated])


R7 =  3   Correlation =  0.003856829479547511


In [9]:
numtraces = trace_array.shape[0]  #Number of traces
numpoints = trace_array.shape[1]  #Number of points in a trace

#Points within which the correlating point lies in 
start_point = 0                   #Starting Point: Point from which the correlating point might lie in
end_point = 5000                  #Ending Point: Point w which the correlating point might lie in

crvs = np.zeros((256, numpoints))
#Hamming Weight Matrix
HW_matrix = np.zeros((numtraces, 256), dtype=np.uint8)

#Stores the correlation value for each key guess
temp = np.zeros(256)

for kguess in range(0, 256):
    
    # Generate the hamming weight for each possible keybyte
    for trace_no in range(numtraces):
        nonce_temp = conversion(nonce_array[trace_no])
        iv = intermediate(nonce_temp, kguess, 12)
        HW_matrix[trace_no, kguess] = HW[iv]

    # correlate the trace with the hamming weights        
    hw = HW_matrix[:, kguess]
    for point in range(start_point, end_point):
        trc = trace_array[:, point]
        crvs[kguess, point] = np.abs(linregress(hw, trc).slope)

    #Finding the point in the trace which has maximum correlation        
    temp[kguess] = np.max(crvs[kguess])

#Finding the key guess that corresponds to maximum correlation
key_estimated = np.argmax(temp)
print("R12 = ",key_estimated, "  Correlation = ", temp[key_estimated])


R12 =  1   Correlation =  0.0020635453486305567


In [10]:
numtraces = trace_array.shape[0]  #Number of traces
numpoints = trace_array.shape[1]  #Number of points in a trace

#Points within which the correlating point lies in 
start_point = 0                   #Starting Point: Point from which the correlating point might lie in
end_point = 5000                  #Ending Point: Point w which the correlating point might lie in

crvs = np.zeros((256, numpoints))
#Hamming Weight Matrix
HW_matrix = np.zeros((numtraces, 256), dtype=np.uint8)

#Stores the correlation value for each key guess
temp = np.zeros(256)

for kguess in range(0, 256):
    
    # Generate the hamming weight for each possible keybyte
    for trace_no in range(numtraces):
        nonce_temp = conversion(nonce_array[trace_no])
        iv = intermediate(nonce_temp, kguess, 13)
        HW_matrix[trace_no, kguess] = HW[iv]

    # correlate the trace with the hamming weights        
    hw = HW_matrix[:, kguess]
    for point in range(start_point, end_point):
        trc = trace_array[:, point]
        crvs[kguess, point] = np.abs(linregress(hw, trc).slope)

    #Finding the point in the trace which has maximum correlation        
    temp[kguess] = np.max(crvs[kguess])

#Finding the key guess that corresponds to maximum correlation
key_estimated = np.argmax(temp)
print("R13 = ",key_estimated, "  Correlation = ", temp[key_estimated])


R13 =  1   Correlation =  0.002523464382928459


In [11]:
numtraces = trace_array.shape[0]  #Number of traces
numpoints = trace_array.shape[1]  #Number of points in a trace

#Points within which the correlating point lies in 
start_point = 0                   #Starting Point: Point from which the correlating point might lie in
end_point = 5000                  #Ending Point: Point w which the correlating point might lie in

crvs = np.zeros((256, numpoints))
#Hamming Weight Matrix
HW_matrix = np.zeros((numtraces, 256), dtype=np.uint8)

#Stores the correlation value for each key guess
temp = np.zeros(256)

for kguess in range(0, 256):
    
    # Generate the hamming weight for each possible keybyte
    for trace_no in range(numtraces):
        nonce_temp = conversion(nonce_array[trace_no])
        iv = intermediate(nonce_temp, kguess, 14)
        HW_matrix[trace_no, kguess] = HW[iv]

    # correlate the trace with the hamming weights        
    hw = HW_matrix[:, kguess]
    for point in range(start_point, end_point):
        trc = trace_array[:, point]
        crvs[kguess, point] = np.abs(linregress(hw, trc).slope)

    #Finding the point in the trace which has maximum correlation        
    temp[kguess] = np.max(crvs[kguess])

#Finding the key guess that corresponds to maximum correlation
key_estimated = np.argmax(temp)
print("R14 = ",key_estimated, "  Correlation = ", temp[key_estimated])


R14 =  118   Correlation =  0.003973172837808248


In [12]:
numtraces = trace_array.shape[0]  #Number of traces
numpoints = trace_array.shape[1]  #Number of points in a trace

#Points within which the correlating point lies in 
start_point = 0                   #Starting Point: Point from which the correlating point might lie in
end_point = 5000                  #Ending Point: Point w which the correlating point might lie in

crvs = np.zeros((256, numpoints))
#Hamming Weight Matrix
HW_matrix = np.zeros((numtraces, 256), dtype=np.uint8)

#Stores the correlation value for each key guess
temp = np.zeros(256)

for kguess in range(0, 256):
    
    # Generate the hamming weight for each possible keybyte
    for trace_no in range(numtraces):
        nonce_temp = conversion(nonce_array[trace_no])
        iv = intermediate(nonce_temp, kguess, 15)
        HW_matrix[trace_no, kguess] = HW[iv]

    # correlate the trace with the hamming weights        
    hw = HW_matrix[:, kguess]
    for point in range(start_point, end_point):
        trc = trace_array[:, point]
        crvs[kguess, point] = np.abs(linregress(hw, trc).slope)

    #Finding the point in the trace which has maximum correlation        
    temp[kguess] = np.max(crvs[kguess])

#Finding the key guess that corresponds to maximum correlation
key_estimated = np.argmax(temp)
print("R15 = ",key_estimated, "  Correlation = ", temp[key_estimated])


R15 =  193   Correlation =  0.0030985375272982196
